In [1]:
import torch
import torch.nn as nn
from einops.layers.torch import Rearrange, Reduce
from torch import Tensor


KeyError: 'gru_hidden_size'

In [2]:
batch_size = 200
seq_len = 30
window_size = 250
# Get a random batch of EEG data
x = torch.rand(batch_size, seq_len, window_size)


In [3]:
from transformiloop.src.models.transformers import TransformiloopFinetune
from transformiloop.src.utils.configs import initialize_config

config = initialize_config('Test')
finetuner = TransformiloopFinetune(config) 

In [4]:
finetuner.cuda()
finetuner(x.cuda(), None).shape

torch.Size([200, 5])

In [5]:
config['seq_len']

138

In [6]:
from torchinfo import summary
summary(
        finetuner,
        input_size=[
            (config['batch_size'], 30, config['window_size']),
            (config['batch_size'], 30-1)
        ],
        dtypes=[torch.float, torch.float, torch.float],
        depth=4,
    )

Layer (type:depth-idx)                             Output Shape              Param #
TransformiloopFinetune                             [200, 5]                  --
├─PatchEmbedding: 1-1                              [200, 138, 64]            --
│    └─Flatten: 2-1                                [200, 7500]               --
│    └─Sequential: 2-2                             [200, 64, 138]            --
│    │    └─Conv1d: 3-1                            [200, 64, 299]            3,264
│    │    └─BatchNorm1d: 3-2                       [200, 64, 299]            128
│    │    └─ELU: 3-3                               [200, 64, 299]            --
│    │    └─AvgPool1d: 3-4                         [200, 64, 138]            --
│    │    └─Dropout: 3-5                           [200, 64, 138]            --
├─Transformer: 1-2                                 [200, 139, 64]            64
│    └─PositionalEncoder: 2-3                      [200, 139, 64]            --
│    └─TransformerEncoder: 2-4 